In [ ]:
# Dependencies to Visualize the model
%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

In [ ]:
# Filepaths, numpy, and Tensorflow
import os
import numpy as np
import tensorflow as tf

In [ ]:
# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Keras Specific
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [ ]:
import pandas as pd

url = "https://dataclass-project4.s3.amazonaws.com/fer2013.csv"
df = pd.read_csv(url)

In [ ]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [ ]:
# Turning the pixels column into a list of arrays, so each image is an array
X_values = df['pixels'].tolist()
X = []
for i in X_values:
  X_list = [int(x) for x in i.split(' ')]
  X_list = np.asarray(X_list)
  X.append(X_list)

In [ ]:
X = np.asarray(X)

In [ ]:
y_df = pd.get_dummies(df['emotion'])
y_df.head()

y = []
for index, row in y_df.iterrows():
    row_array = np.array(row.values)
    y.append(row_array)
y = np.asarray(y)    

In [ ]:
# Splitting the data into training and testing sets
# Before the model creation, you need to reshape X
# Assuming that you have grayscale images of size 48x48
X = X.reshape(-1, 48, 48, 1)  # reshaping into (num_images, 48, 48, 1)

# Now split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert lists to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Rescale the images
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255


In [ ]:
# Creating the model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dense, Dropout, Activation, Flatten
model = Sequential()
model.add(Conv2D(64,kernel_size=(3, 3), activation='relu', input_shape=(48,48,1), data_format="channels_last"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(64*2,kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(64*4,kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(64*8,kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(8*64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(4*64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*64, activation='relu'))
model.add(Dropout(0.4))

#output layer
model.add(Dense(units=7, activation='softmax'))

In [ ]:
# Compiling the model
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
# Fitting the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
898/898 - 21s - loss: 1.7509 - accuracy: 0.3690 - 21s/epoch - 24ms/step
Epoch 2/100
898/898 - 9s - loss: 1.3519 - accuracy: 0.4935 - 9s/epoch - 10ms/step
Epoch 3/100
898/898 - 9s - loss: 1.2299 - accuracy: 0.5390 - 9s/epoch - 10ms/step
Epoch 4/100
898/898 - 9s - loss: 1.1486 - accuracy: 0.5699 - 9s/epoch - 10ms/step
Epoch 5/100
898/898 - 9s - loss: 1.0925 - accuracy: 0.5907 - 9s/epoch - 10ms/step
Epoch 6/100
898/898 - 9s - loss: 1.0268 - accuracy: 0.6155 - 9s/epoch - 10ms/step
Epoch 7/100
898/898 - 9s - loss: 0.9568 - accuracy: 0.6424 - 9s/epoch - 10ms/step
Epoch 8/100
898/898 - 9s - loss: 0.8915 - accuracy: 0.6701 - 9s/epoch - 10ms/step
Epoch 9/100
898/898 - 9s - loss: 0.8332 - accuracy: 0.6939 - 9s/epoch - 10ms/step
Epoch 10/100
898/898 - 9s - loss: 0.7571 - accuracy: 0.7179 - 9s/epoch - 10ms/step
Epoch 11/100
898/898 - 9s - loss: 0.6957 - accuracy: 0.7426 - 9s/epoch - 10ms/step
Epoch 12/100
898/898 - 9s - loss: 0.6428 - accuracy: 0.7654 - 9s/epoch - 10ms/step
Epoch 13/10

In [ ]:
# Evaluating the model
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

# First time - Loss: 1.9519965648651123, Accuracy: 0.6375035047531128
# Dropout 20% on each layer
# Second time - Loss: 1.2198774814605713, Accuracy: 0.6517135500907898
# Dropout 40% on each layer
# Third time - Loss: 1.1788020133972168, Accuracy: 0.6213430166244507
# Added three dense layers

225/225 - 1s - loss: 1.9520 - accuracy: 0.6375 - 1s/epoch - 5ms/step
Normal Neural Network - Loss: 1.9519965648651123, Accuracy: 0.6375035047531128


In [ ]:
# Saving the model
model.save("emotion_model.h5")

In [ ]:
# Loading the model
from tensorflow.keras.models import load_model
emotion_model = load_model('emotion_model.h5')

In [ ]:
# Making predictions
predicted = emotion_model.predict(X_test_scaled)
predicted = np.argmax(predicted, axis=1)
predicted

225/225 [==============================] - 1s 2ms/step


array([3, 3, 6, ..., 0, 5, 2])

In [ ]:
# Creating a dataframe of the predictions
predicted_df = pd.DataFrame(predicted)
predicted_df = predicted_df.rename(columns={0: "Predicted"})
predicted_df.head()

,Predicted
0,3
1,3
2,6
3,4
4,6


In [ ]:
# Creating a dataframe of the actual values
y_test_actual = np.argmax(y_test, axis=1)

# Creating a dataframe of the actual values
y_test_df = pd.DataFrame(y_test_actual, columns=["Actual"])

y_test_df.head()

,Actual
0,3
1,6
2,0
3,4
4,0


In [ ]:
# Merging the two dataframes
merged_df = pd.merge(y_test_df, predicted_df, left_index=True, right_index=True)
merged_df.head()

,Actual,Predicted
0,3,3
1,6,3
2,0,6
3,4,4
4,0,6


In [ ]:
# Creating a dataframe of the emotions
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
emotions_df = pd.DataFrame(emotions)
emotions_df = emotions_df.rename(columns={0: "Emotions"})
emotions_df.head()

,Emotions
0,Angry
1,Disgust
2,Fear
3,Happy
4,Sad


In [ ]:
# Merging the emotions dataframe with the merged dataframe
merged_df = pd.merge(merged_df, emotions_df, left_on='Actual', right_index=True)
merged_df.head()

,Actual,Predicted,Emotions
0,3,3,Happy
11,3,3,Happy
25,3,3,Happy
26,3,3,Happy
31,3,3,Happy


In [ ]:
# Merging the emotions dataframe with the merged dataframe
merged_df = pd.merge(merged_df, emotions_df, left_on='Predicted', right_index=True)
merged_df.head()

,Actual,Predicted,Emotions_x,Emotions_y
0,3,3,Happy,Happy
11,3,3,Happy,Happy
25,3,3,Happy,Happy
26,3,3,Happy,Happy
31,3,3,Happy,Happy
